# Chapter 11: SOC 분석가 에이전트

이 노트북에서는 보안 운영 센터(SOC) 분석가 역할의 AI 에이전트를 구축합니다.

## 주요 내용
- 위협 인텔리전스 조회
- 보안 로그 분석
- 사고 분류 및 대응

⚠️ **참고**: Traceloop/Loki 관측성 기능은 Colab 호환을 위해 제거되었습니다.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/notebook/ch11_soc_analyst_agent.ipynb)


In [ ]:
!pip install -q langchain langchain-openai langgraph python-dotenv


In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
    pass
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"


In [ ]:
import json
import operator
from typing import Annotated, Sequence, TypedDict, Optional
from langchain.chat_models import init_chat_model
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph

# 보안 분석 도구 정의
@tool
def lookup_threat_intel(indicator: str, type: str) -> str:
    """IP, 파일 해시, URL에 대한 위협 인텔리전스를 조회합니다."""
    print(f"[TOOL] lookup_threat_intel(indicator={indicator}, type={type})")
    return "threat_intel_retrieved: 악성 IP로 확인됨"

@tool
def query_logs(query: str, log_index: str) -> str:
    """보안 로그를 검색합니다."""
    print(f"[TOOL] query_logs(query={query}, log_index={log_index})")
    return "log_query_executed: 5건의 의심스러운 활동 발견"

@tool
def triage_incident(incident_id: str, decision: str, reason: str) -> str:
    """사고를 True Positive/False Positive로 분류합니다."""
    print(f"[TOOL] triage_incident(incident_id={incident_id}, decision={decision})")
    return f"incident_triaged: {decision}"

@tool
def isolate_host(host_id: str, reason: str) -> str:
    """손상된 호스트를 격리합니다."""
    print(f"[TOOL] isolate_host(host_id={host_id}, reason={reason})")
    return "host_isolated"

TOOLS = [lookup_threat_intel, query_logs, triage_incident, isolate_host]
print("✅ SOC 분석 도구 정의 완료")


In [ ]:
# SOC 분석가 에이전트 구성
llm = init_chat_model(model="gpt-5-mini").bind_tools(TOOLS)

class AgentState(TypedDict):
    incident: Optional[dict]
    messages: Annotated[Sequence[BaseMessage], operator.add]

def call_model(state: AgentState):
    history = state["messages"]
    incident = state.get("incident", {"incident_id": "UNKNOWN", "severity": "medium"})
    
    system_prompt = f"""당신은 숙련된 SOC 분석가입니다.

전문 분야:
- 위협 인텔리전스 분석 및 IOC 조사
- 보안 로그 분석 및 상관관계 분석
- 사고 분류 및 선별 (True Positive/False Positive)
- 사고 봉쇄 및 대응

조사 방법:
1) 보안 경고를 분석하고 지표를 수집합니다
2) lookup_threat_intel로 IP, 해시, URL을 조사합니다
3) query_logs로 관련 로그를 검색합니다
4) triage_incident로 결과를 분류합니다
5) 필요시 isolate_host로 호스트를 격리합니다

사고(INCIDENT): {json.dumps(incident, ensure_ascii=False)}"""

    full = [SystemMessage(content=system_prompt)] + history
    first = llm.invoke(full)
    messages = [first]

    if getattr(first, "tool_calls", None):
        for tc in first.tool_calls:
            fn = next(t for t in TOOLS if t.name == tc['name'])
            out = fn.invoke(tc["args"])
            messages.append(ToolMessage(content=str(out), tool_call_id=tc["id"]))
        second = llm.invoke(full + messages)
        messages.append(second)

    return {"messages": messages}

def construct_graph():
    g = StateGraph(AgentState)
    g.add_node("assistant", call_model)
    g.set_entry_point("assistant")
    return g.compile()

graph = construct_graph()
print("✅ SOC 분석가 에이전트 구성 완료")


In [ ]:
# 테스트 실행
example = {"incident_id": "INC-12345", "severity": "high", "type": "suspicious_login"}
convo = [HumanMessage(content='경고가 발생했습니다: "IP 203.0.113.45에서 관리자 계정으로 의심스러운 로그인 시도." 어떻게 해야 하나요?')]

result = graph.invoke({"incident": example, "messages": convo})

print("\n" + "=" * 60)
print("🔒 SOC 분석가 에이전트 응답")
print("=" * 60)
for m in result["messages"]:
    print(f"\n{m.type}: {m.content}")
